In [1]:
#YouTube Crawling & DB Insert

In [ ]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from urllib.parse import quote
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time
import random
import cx_Oracle
import re
from selenium.webdriver.chrome.service import Service
from openpyxl import Workbook

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
# driver = webdriver.Chrome('C:/Users/DY/.wdm/drivers/chromedriver/win32/113.0.5672.63/chromedriver.exe', chrome_options=chrome_options)
driver = webdriver.Chrome(chrome_options=chrome_options)
def db_delete_youtube():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    print('')
    print('DB connect 성공!!')
    print('')

    cs = conn.cursor()
    
    sql = "DELETE FROM YOUTUBE_TABLE"

    cs.execute(sql)
    
    deleted_rows = cs.rowcount
    print(f"{deleted_rows} rows deleted")
    
    cs.close()
    conn.commit()
    conn.close()

def db_delete_comment():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    print('')
    print('DB connect 성공!!')
    print('')

    cs = conn.cursor()
    
    sql = "DELETE FROM YC_TABLE"

    cs.execute(sql)
    
    deleted_rows = cs.rowcount
    print(f"{deleted_rows} rows deleted")
    
    cs.close()
    conn.commit()
    conn.close()

def db_insert(title,views, upload_date, link,extract_date): # DB 저장 함수(크롤링 추출 자료 INSET)
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    print('')
    print('DB connect 성공!!')
    print('')

    cs = conn.cursor()
    
    pattern = r'^조회수 ([0-9.]+)([천만]?)회$'
    replacement = r'\1\2'
    views = re.sub(pattern, replacement, views)
    extract_date=extract_date[:len(extract_date)-2]

    if(upload_date[0]=='스'):
        upload_date=upload_date[8:]
        
    upload_date=extract_date
    
    
    
    sql = "INSERT INTO YOUTUBE_TABLE (TITLE, VIEWS, UPLOAD_DATE, LINK, EXTRACT_DATE) VALUES (:1, :2, :3, :4, :5)"

  

    cs.execute(sql, (title, views, upload_date, link, extract_date))
    
    print("[%d]건 INSERT 완료" % 30)
    cs.close()
    conn.commit()
    conn.close()



def cleansing_sentence(input_string):
    string_pattern = re.compile(r'[^ㄱ-힣 0-9 a-z A-Z]')
    cleansing_string = string_pattern.sub('', input_string)
    return cleansing_string

def scroll():
    try:       
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

def scroll2():
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    for i in range(2):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        #time.sleep(1.5)
        time.sleep(3)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(3)

def closeAD():
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass


def get_data():
    #원하는 검색어 입력받음
    query = input('원하는 검색을 입력하시오:')

    #URL 지정
    URL = f"https://www.youtube.com/results?search_query={quote(query)}"

    #URL 실행
    driver.get(URL)

    #필터 버튼 클릭
    driver.find_element(By.XPATH, '//*[@id="container"]/ytd-toggle-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
    sleep(1)

    #이번 달
    upload_date = driver.find_element(By.XPATH,'//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]')
    click_list_1 = upload_date.find_elements(By.ID,"endpoint")
    click_list_1[3].click()
    sleep(1)

    driver.find_element(By.XPATH, '//*[@id="container"]/ytd-toggle-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
    length = driver.find_element(By.XPATH,'//*[@id="collapse-content"]/ytd-search-filter-group-renderer[3]')
    click_list_2 = length.find_elements(By.ID,"endpoint")
    click_list_2[1].click()
    sleep(1)
    # 무한 스크롤 함수 실행
    scroll()

    # 페이지 소스 추출
    html_source = driver.page_source
    soup_source = BeautifulSoup(html_source, 'html.parser')

    # 콘텐츠 모든 정보
    content_total = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
    content_record_src = soup_source.select("span.inline-metadata-item")
    # 콘텐츠 제목만 추출
    content_total_title = list(map(lambda data: data.get_text().replace("\n", ""), content_total))
    content_record = list(map(lambda data: data.get_text().replace("\n", ""), content_record_src))
    # 콘텐츠 링크만 추출
    content_total_link = list(map(lambda data: "https://youtube.com" + data["href"], content_total))

    i = 0
    for title in content_total_title:
        title = cleansing_sentence(content_total_title[i])
        link = content_total_link[i]
        view = content_record[2*i]
        upload_date = content_record[2*i + 1]
        current_date = datetime.datetime.now()
        extract_date = current_date.strftime('%Y%m%d')
        db_insert(title,view, upload_date, link,extract_date)
        print("제목 : ",title)
        print("조회수 : ",view)
        print("날짜 : ",upload_date)
        print("링크 : ",link)
        print("추출 날:",extract_date)
        print("-----------------------------------------------")
        i = i + 1

def comment():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()
    ## sql = "SELECT LINK FROM YOUTUBE_TABLE"
    sql = "SELECT LINK FROM YOUTUBE_TABLE FETCH FIRST 100 ROWS ONLY"
    sql_insert = "INSERT INTO YC_TABLE (DATA_LINK, DATA_COMMENT) VALUES (:1,:2)"
    cs.execute(sql)
    #테이블 링크 가져오기
    links = cs.fetchall()
    for link in links:
        URL = link[0]
            #URL 실행
        driver.get(URL)

        wb = Workbook(write_only=True)
        ws = wb.create_sheet()

        driver.implicitly_wait(5)

        time.sleep(1.5)

        driver.execute_script("window.scrollTo(0, 800)")
        time.sleep(3)
        # 스크롤
        scroll2()
        closeAD()

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        comment_list = soup.select("yt-formatted-string#content-text")

        
        comment_final = []
        #----------------------유튜브 댓글 가져오기-------------------------------------------------#
        for i in range(len(comment_list)):
            temp_comment = comment_list[i].text
            temp_comment = temp_comment.replace('\n', '')
            temp_comment = temp_comment.replace('\t', '')
            temp_comment = temp_comment.replace('    ', '')
            final_comment = cleansing_sentence(temp_comment)
        
            cs.execute(sql_insert, (str(URL), str(final_comment)))
        #--------------------------------------------------------------------------------------------#
    cs.close()
    conn.commit()
    conn.close()

if __name__ == '__main__':
    cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient-basic-windows.x64-21.9.0.0.0dbru\instantclient_21_9")
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--start-maximized')
    driver = webdriver.Chrome(chrome_options=chrome_options)
    service = Service(ChromeDriverManager().install())
    db_delete_youtube()
    db_delete_comment()
    get_data()
    comment()